<h1 style="color:brown;">  Welcome to Modeling</h1> 

### Import packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import scipy.stats as stats
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Lesson plan - linear regression & general approach to modelling
1. The story/question 
2. The aim
3. Assumptions and constraints
4. Optimization
5. Metric
6. Interpretation - if time permits


What is a model?

### How are the following factors related?  

##### Example I : 
Dan went out with friends, drank, drove his car and crashed - he's ok. 
Joe went out with friends, didn't drink, also drove his car and crashed as well.

![ChessUrl](https://media.giphy.com/media/hDJo64zI3T6da/giphy.gif "chess")

##### Example II: 
With every passing year, CO2 levels increase 

![Co2](https://media.giphy.com/media/VTQGgVfLeTL32/giphy.gif)

##### Example III: 
The more ice cream people eat, the more drownings we have.

![Ice-cream](https://media.giphy.com/media/3o6gDREoETcSrm9Wog/giphy.gif)

![Lumber](https://media.giphy.com/media/UxTZDNv0Zej4s/giphy.gif)

##### Example V: 
An experiment was done with different concentrations of toxic substance to quantify effect on tumor incidence

![tumor](https://media.giphy.com/media/3o85xHEY82v0ZEpAwo/giphy.gif)

### 2. What is the aim of each example?  take 5 minutes to tag each example !

##### Why are causal statements tricky? 

### 3. Why do we even need a model to find patterns in the data? 

![biased tall](https://media.giphy.com/media/l0MYrguDYf6uvscak/giphy.gif)

In [ ]:
##### Enter your answer here


<h2 style="color:purple;"> Assumption I: linear relation between X & Y </h2>

##### We are going to concentrate on drunken Dan for our hands on example

##### Load the data

In [ ]:
sns.get_dataset_names()

In [ ]:
car_crashes_df = sns.load_dataset('car_crashes')

In [ ]:
car_crashes_df.head()

### a. What are possible simple linear regression models one might want to run here?  

In [1]:
# Choose specifically columns of total and alcohol
# give this df a different name than the original


### b. Can you think of two quick statistical ways to check the intuition that alcohol 
### and car crashing index are linearly associated?

Hint: $\frac{cov(x, y)}{\sqrt{var(X)var(Y)}}$

In [ ]:
coef_of_deter = stats.pearsonr(car_crashes_df['alcohol'], car_crashes_df['total'])[0]


In [ ]:
plt.scatter(car_crashes_df['alcohol'], car_crashes_df['total'])
plt.show()

### c.  let's calculate the correlation of alcohol and total crash index? 

In [ ]:
### Calculate the correlation coef with stats model
stats_model_corr = stats.pearsonr(car_crashes_df['alcohol'],
                                  car_crashes_df['total'])[0]

In [ ]:
### Calculate the correlation coef with numpy - pay attention that sample variance is different from population variance !
correlation = (np.cov(car_crashes_df['alcohol'],
                      car_crashes_df['total'])[1, 0] /
               np.sqrt(np.var(car_crashes_df['alcohol'], ddof = 1)
                       * np.var(car_crashes_df['total'], ddof = 1)) 

##### Sample variance is biased 
- https://en.wikipedia.org/wiki/Bessel%27s_correction#Proof_of_correctness_-_Alternate_3
- much more rigorous math (proof) : https://mycourses.aalto.fi/pluginfile.php/364838/mod_resource/content/1/Proof%20that%20Sample%20Variance%20is%20Unbiased.pdf 

### d. Calculate both the covariance and correlation of the pairs: 
 - alcohol and not_distracted
 - alcohol and no_previous

In [ ]:
# get covariance matrix (variance on the diagonal and covariance otherwise)
cov_mat_alc_not_dist = np.cov(car_crashes_df['alcohol'],
                              car_crashes_df['not_distracted'])

In [ ]:
# get only the value of the covariance between alcohol and no_previous
cov_val_alc_no_prev = np.cov(car_crashes_df['alcohol'],
                             car_crashes_df['no_previous'])[1, 0]

In [ ]:
# now let's get the coeficient of determination of both pairs
stats_model_corr_alc_not_dist = stats.pearsonr(car_crashes_df['alcohol'],
                                               car_crashes_df['not_distracted'])[0]

In [ ]:
stats_model_corr_alc_no_prev = stats.pearsonr(car_crashes_df['alcohol'],
                                              car_crashes_df['no_previous'])[0]

##### Do you understand the results? 
Covariance results can be misleading since they take into account measurements in different units
and thus are hard to compare (think of cm versus meters)

### If you were to choose only one number (constant) for the model what would it be? 
### Talk with a peer and calculate this number from the data set.

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(car_crashes_df['alcohol'], car_crashes_df['total'])
plt.axhline(y=20, color = 'green', label = 'h2')
plt.axhline(y=12, color = 'red', label = 'h1')
plt.axhline(y=15, color = 'brown', label = 'h3')
plt.legend()
plt.xlabel('Alcohol consumption')
plt.ylabel('Crash severity index')
plt.title('Which line is the optimal constant line? ')
plt.show()

![confused](https://media.giphy.com/media/3o7btPCcdNniyf0ArS/giphy.gif)

### Can we tilt the line to get a better fit? 

### 4. Optimal

##### Deciding on the optimal line requires us to define what is optimal - this is similar to all learning algorithms 

Our goal is to minimise the loss function (errors squared)
$MSE = \frac{1}{n} \sum (actual - predicted) \square{2}$

##### a. Calculate the sum of errors from your model? Do you understand why you got this result ? 
##### Will it be the same for non constant number? https://autarkaw.org/2017/07/06/sum-of-the-residuals-for-the-linear-regression-model-is-zero/

In [ ]:
sum_residuals  = sum(car_crashes_df['total'] 
                     - np.mean(car_crashes_df['total']))

In [ ]:
##### Please take 3 min to explore the ols method of smf package
?smf.ols

<h2 style="color:purple;">  We can tilt it! now our goal is to find a line such that </h2>


$\hat y_i  = ax_i + b$ 

### 5. Metrics

#### how do I know if it is really a good model ? 

![e-commerce](https://media.giphy.com/media/3o6ZsYYi4MmCcr9KN2/giphy.gif)

In [ ]:
### Take 2 minutes to plot (lmplot) with seaborn 'alcohol' as your explanatory variable and 'ins_premium' as your target

In [ ]:
sns.lmplot('alcohol', 'ins_premium', data=car_crashes_df)

### For linear regression specifically we can talk about R^2

$ R^2 = 1 - \frac {\sum (y_i - \hat y)^2} {\sum (y_i - y_{mean})^2}$

In [ ]:
### Take 3 min to calculate the R^2 using stats.linregress

In [ ]:
r_squared = stats.linregress(alcohol_df['alcohol'],
                             alcohol_df['total']).rvalue ** 2

##### Should we always count on R^2? 

### Are we done? 

![](https://media.giphy.com/media/5bofVYmYAEDVCYERcf/giphy.gif)

### We need more assumptions..  

<h2 style="color:purple;"> Assumption II: errors are independent </h2>

<h2 style="color:purple;"> Assumption III: errors are normally distributed </h2>

<h2 style="color:purple;"> Assumption IV: errors have mean 0 and std sigma (homoscedastic)</h2>

![](https://i.stack.imgur.com/yEqcg.png)

### How can we check model assumptions? (these are only few suggestions)

1. scatter plot  - check linearity, constant variance and outliers 

2. standardized residuals vs. explanatory variable - should look like a cloud because of independence assumption

3. predicted values vs. standardized residuals - should look like a cloud (comes out from the math)

4. Histograms for standardized residuals

5. Q-Q plots

What are standardized residuals [by Dr Giampiero Marra] : 
    Residuals and standardized residuals. The residuals are the deviations of the observations of their “ideal” value according to the model.
    They can be interpreted as estimators of the errors ei (and are therefore denoted by eˆi). This suggests that the residuals can be used to assess the model assumptions about the errors. The standard- ised residuals are residuals divided by their estimated standard deviations, so that the standardized residuals should have a standard deviation of 1, which helps assessing their size. Too many standardised residuals with magnitude greater than 2 suggests that the error distribution has heavier tails than a normal distribution, but note that about 5% of standardised residuals are expected to be larger than 2 in magnitude even if the errors are normally distributed (remember that 2 ≈ 1.96 = 97.5% quantile of the standard normal distribution).

### Let's take a go and try to check the assumptions of linear regression
### and then run a model

##### 1. Check the linear regression assumptions (use histograms, Q-Q plots, scatter plots, residual plots)
##### 2. Write down the equation you got. How do you interpret it? 
##### 3. Write down the confidence intervals for the intercept and slope
##### 4. Are any of the parameters insignificant? 
##### 5. Add a column to data frame with the prediction (according to the equation you got)

### Check model assumptions 

In [ ]:
plt.scatter(car_crashes_df['alcohol'],
            car_crashes_df['total'])

In [ ]:
resid1 = model.resid
fig = sm.graphics.qqplot(resid1,
                         dist=stats.norm,
                         line='45',
                         fit=True)
fig.show()

In [ ]:
plt.hist(resid1)
plt.show()

### Construct the model

In [ ]:
model = smf.ols(formula='alcohol~total',
                data=car_crashes_df).fit()

In [ ]:
model.summary()

In [ ]:
## Plot linear regression line with confidence band


In [ ]:
sns.lmplot('alcohol', 'total', data=car_crashes_df)